# Lab 2
## Medical Patient No Show

### Team members: Luay Dajani, Dana Geislinger, Chris Morgan, Caroll Rodriguez
##### Github - https://github.com/cdmorgan103/7331DataMiningNoShow

MSDS 7331, 10/28/2018

In [1]:
import numpy as np
from __future__ import print_function
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
from pprint import pprint
from IPython.display import display
import seaborn as sns; sns.set(font_scale=1.2)
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit 
import datetime 
from sklearn.model_selection import TimeSeriesSplit
import seaborn as sns

# Hide deprecation warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Load the data into variable 'df' from pickled object
from funcs import load_df
df = load_df()

## Data Preparation Part 1
#### [10 points] Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.

**Meetings Notes: 10/24 ** Chris - copy from MiniLab
Variable to remove - AppointmentID & PatientID


SciKitLearn models have difficulty working with complex objects such as datetime.datetime and datetime.time. To account for this, we will store all dates as ordinal values (the number of days since 1/1/1) and all time values as the number of elapsed seconds in the day (1:30PM = 13 * 3600 + 30 * 60).

In [2]:
# Convert date columns to ordinal integer values (days since 1/1/1)
dt_ord = lambda dt: dt.toordinal()
if 'ScheduledDay' in df:
    df['ScheduledDayOrdinal'] = df['ScheduledDay'].apply(dt_ord)
if 'AppointmentDay' in df:
    df['AppointmentDayOrdinal'] = df['AppointmentDay'].apply(dt_ord)

# Convert time values to seconds (total seconds since start of day)
to_secs = lambda t: t.hour * 3600 + t.minute * 60 + t.second
if 'ScheduledTime' in df:
    df['ScheduledTimeSeconds'] = df['ScheduledTime'].apply(to_secs)

In [3]:
df[['ScheduledDayOrdinal', 'ScheduledTimeSeconds', 'AppointmentDayOrdinal']].head()

,ScheduledDayOrdinal,ScheduledTimeSeconds,AppointmentDayOrdinal
0,736083,67088,736083
1,736083,58107,736083
2,736083,58744,736083
3,736083,62971,736083
4,736083,58043,736083


In [4]:
#Remove attributes not usefull
del df['PatientId']
del df['AppointmentID']

# perform one-hot encoding of the categorical data 
tmp_df = pd.get_dummies(df.Handicap,prefix='Handicap')
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

tmp_df = pd.get_dummies(df.AppointmentDOW,prefix='AppointmentDOW')
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

tmp_df = pd.get_dummies(df.ScheduledDOW,prefix='ScheduledDOW')
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

tmp_df = pd.get_dummies(df.Neighbourhood,prefix='Neighbourhood')
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

tmp_df = pd.get_dummies(df.age_range,prefix='age_range')
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

# replace the current Gender attribute with something slightly more intuitive and readable
df['IsMale'] = df.Gender=='M' 
df.IsMale = df.IsMale.astype(np.int)

# Now let's clean up the dataset
if 'Gender' in df:
    del df['Gender'] # if 'Sex' column still exists, delete it (as we created an ismale column)
    
if 'Handicap' in df:    
    del df['Handicap'] # get rid of the original category as it is now one-hot encoded
    
if 'ScheduledDOW' in df:    
    del df['ScheduledDOW'] # get rid of the original category as it is now one-hot encoded
    
if 'AppointmentDOW' in df:    
    del df['AppointmentDOW'] # get rid of the original category as it is now one-hot encoded

if 'Neighbourhood' in df:    
    del df['Neighbourhood'] # get rid of the original category as it is now one-hot encoded

if 'age_range' in df:
    del df['age_range']

if 'AppointmentID' in df:
    del df['AppointmentID']
    
if 'ScheduledDay' in df:
    del df['ScheduledDay']
    
if 'ScheduledTime' in df:
    del df['ScheduledTime']

if 'AppointmentDay' in df:
    del df['AppointmentDay']

# Get an overview of the raw data
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Columns: 115 entries, Age to IsMale
dtypes: bool(1), int32(1), int64(10), uint8(103)
memory usage: 19.8 MB


In [5]:
print(df.dtypes)

Age                               int64
Scholarship                       int64
Hypertension                      int64
Diabetes                          int64
Alcoholism                        int64
SMSReceived                       int64
NoShow                             bool
DaysInAdvance                     int64
ScheduledDayOrdinal               int64
AppointmentDayOrdinal             int64
ScheduledTimeSeconds              int64
Handicap_0                        uint8
Handicap_1                        uint8
Handicap_2                        uint8
Handicap_3                        uint8
Handicap_4                        uint8
AppointmentDOW_Monday             uint8
AppointmentDOW_Tuesday            uint8
AppointmentDOW_Wednesday          uint8
AppointmentDOW_Thursday           uint8
AppointmentDOW_Friday             uint8
AppointmentDOW_Saturday           uint8
AppointmentDOW_Sunday             uint8
ScheduledDOW_Monday               uint8
ScheduledDOW_Tuesday              uint8


In [6]:
df.describe()

,Age,Scholarship,Hypertension,Diabetes,Alcoholism,SMSReceived,DaysInAdvance,ScheduledDayOrdinal,AppointmentDayOrdinal,ScheduledTimeSeconds,...,Neighbourhood_SÃO CRISTÓVÃO,Neighbourhood_SÃO JOSÉ,Neighbourhood_SÃO PEDRO,Neighbourhood_TABUAZEIRO,Neighbourhood_UNIVERSITÁRIO,Neighbourhood_VILA RUBIM,age_range_child,age_range_adult,age_range_senior,IsMale
count,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,...,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,37.089218,0.098266,0.197246,0.071865,0.030400,0.321026,10.183792,736092.856370,736103.040162,40557.666643,...,0.016611,0.017887,0.022148,0.028337,0.001375,0.007699,0.234060,0.645598,0.120342,0.350023
std,23.109921,0.297675,0.397921,0.258265,0.171686,0.466873,15.254924,19.140133,12.189325,11578.064436,...,0.127811,0.132541,0.147167,0.165934,0.037059,0.087409,0.423412,0.478334,0.325362,0.476979
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,735912.000000,736083.000000,22176.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,736083.000000,736093.000000,30338.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,736094.000000,736102.000000,37983.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,55.000000,0.000000,0.000000,0.000000,0.000000,1.000000,15.000000,736104.000000,736115.000000,50382.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,115.000000,1.000000,1.000000,1.000000,1.000000,1.000000,179.000000,736123.000000,736123.000000,77255.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
#simple export of final dataset
df.to_csv("./data/df.csv", sep=',')

## Data Preparation Part 2
#### [5 points] Describe the ﬁnal dataset that is used for classiﬁcation/regression (include a description of any newly formed variables you created).
**Meetings Notes 10/24:** Chris - copy from MiniLab

**Meeting Notes: ????** Luay wants add a field that ranks the patient's no show history. If they showed up twice and didn't show up 1 would be 33%. Create a temp dataset to group Patient and determine the # of no shows then merge the dataset and build the ranking. 

https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9



In [8]:
max(df['AppointmentDayOrdinal']) - ((max(df['AppointmentDayOrdinal']) - min(df['AppointmentDayOrdinal']))*.3)

736111.0

## Modeling and Evaluation 1

#### [10 points] Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.

**Meetings Notes 10/24:** Chris - copy from MiniLab

## Modeling and Evaluation 2	

#### [10 points]	Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.

**Meeting Notes:** Dana - to figure out error with SMOTE
Caroll to run CV on Training Set and evaluate the accuracy of Test

##### Partioning Data
We want to forecast future NoShows based on Appointment dates. We take the data we have and build a fixed training period in the past up to the last month. The last month of data will be used as testing dataset. Before we deploy the model we will rerun the models utilizing the entire dataset to predict the future outcomes.
This approach fits our dataset since we there is correlation between the appoointme date and whether or not they no-show.

#### Task 1(No-Show)

In [22]:
splitPerc = .3 # percentage

In [24]:
#Testing set --- hold 1 month of data out as test set based on AppointmentDay
df_test = df.copy()
# Calculation to find the ordinal value of AppointmentDay to split data at
X_test = df_test[df['AppointmentDayOrdinal'] >=  (max(df['AppointmentDayOrdinal'])- 
             (max(df['AppointmentDayOrdinal'])-(min(df['AppointmentDayOrdinal'])))* splitPerc)]
y_test = X_test['NoShow'].values # get the labels we want.

if 'NoShow' in X_test:
    del X_test['NoShow'] # get rid of the class label

#Training set -- all records less than appointment date 5/1/2016
df_train = df.copy() 
                 
X_train = df_train[df['AppointmentDayOrdinal'] < (max(df['AppointmentDayOrdinal'])- 
             (max(df['AppointmentDayOrdinal'])-(min(df['AppointmentDayOrdinal'])))* splitPerc)]
y_train = X_train['NoShow'].values # get the labels we want.
del X_train['NoShow'] # get rid of the class label

if 'NoShow' in X_train:
    del X_train['NoShow'] # get rid of the class label

#All data
df_tmp = df.copy()
X = df_tmp
y = df_tmp['NoShow']
if 'NoShow' in X:
    del X['NoShow'] # get rid of the class label

In [25]:
num_cv_iterations = 10
num_instances = len(y)
seed = 123456789
cv_object = TimeSeriesSplit(n_splits=num_cv_iterations,
                         max_train_size= 0.7,
                         )
                         
print(cv_object)

TimeSeriesSplit(max_train_size=0.7, n_splits=10)


In [26]:
print ('Number of Prediciting Features: ', X.shape)
print ('Number of Response: ', y.shape)
print ('Appointment Date Range (Trainig Set): ', min(X_train['AppointmentDayOrdinal']) ," - ",  max(X_train['AppointmentDayOrdinal']))
print ('Number of Training Records: ', "Predictors: ", X_train.shape, "Response:" ,y_train.shape)
print ('Number of Test Records: ', "Predictors: ",X_test.shape, "Response:" ,y_test.shape)
print ('Appointment Date Range (Test Set): ', min(X_test['AppointmentDayOrdinal']) ," - ",  max(X_test['AppointmentDayOrdinal']))

Number of Prediciting Features:  (110527, 114)
Number of Response:  (110527,)
Appointment Date Range (Trainig Set):  736083  -  736109
Number of Training Records:  Predictors:  (75283, 114) Response: (75283,)
Number of Test Records:  Predictors:  (35244, 114) Response: (35244,)
Appointment Date Range (Test Set):  736114  -  736123


#### Grid Search
<p>A grid search will help determine the optimal parameters to pass to the logistic regression function. Finding the optimal parameters will help with model prediction.</p>
<p>Some parameters are being selected as the only option due to the type of dataset. We will try using a solver with a default parameter 'lbfgs' for binomial problems, although sag and saga are faster for larger datasets.</p>
<p>The multi-class parameter is set to the default of 'ovr' because we have a binary problem.

In [28]:
from sklearn.model_selection import cross_validate

def EvaluateClassifierEstimator(classifierEstimator, X, y, cv):
   
    #Perform cross validation 
    scores = cross_validate(classifierEstimator, X, y, scoring=['accuracy','precision','recall']
                            , cv=cv_object, return_train_score=True)

    Accavg = scores['test_accuracy'].mean()
    Preavg = scores['test_precision'].mean()
    Recavg = scores['test_recall'].mean()

    print_str = "The average accuracy for all cv folds is: \t\t\t {Accavg:.5}"
    print_str2 = "The average precision for all cv folds is: \t\t\t {Preavg:.5}"
    print_str3 = "The average recall for all cv folds is: \t\t\t {Recavg:.5}"

    print(print_str.format(Accavg=Accavg))
    print(print_str2.format(Preavg=Preavg))
    print(print_str3.format(Recavg=Recavg))
    print('*********************************************************')

    print('Cross Validation Fold Mean Error Scores')
    scoresResults = pd.DataFrame()
    scoresResults['Accuracy'] = scores['test_accuracy']
    scoresResults['Precision'] = scores['test_precision']
    scoresResults['Recall'] = scores['test_recall']

    return scoresResults

def EvaluateClassifierEstimator2(classifierEstimator, X, y, cv_object):
    
    #Perform cross validation 
    from sklearn.model_selection import cross_val_predict
    predictions = cross_val_predict(classifierEstimator, X, y, cv=cv_object)
    
    #model evaluation 
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    
    #pass true test set values and predictions to classification_report
    classReport = classification_report(y,predictions)
    confMat = confusion_matrix(y,predictions)
    acc = accuracy_score(y,predictions)
    
    print(classReport)
    print(confMat)
    print(acc)

Upon researching the possible parameters to LogisticRegression from sklearn, we decided that the most important parameters of interest were C (the inverse of regularization strength, which must be a positive floating point value) and max_iter (the maximum number of iterations for the lbfgs solver to use). The default values for these parameters are 1.0 and 100 respectively, and we have included a range of potential values for these parameters that might help improve our model accuracy.

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

regEstimator = LogisticRegression()

parameters = {
             }

#Create a grid search object using the  
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=regEstimator
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv_object # KFolds
                   , scoring='accuracy'
                            )
#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


TypeError: slice indices must be integers or None or have an __index__ method

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

regEstimator = RandomForestClassifier()

parameters = {
             }

#Create a grid search object using the  
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=regEstimator
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv_object # KFolds
                   , scoring='accuracy'
                            )
#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


TypeError: slice indices must be integers or None or have an __index__ method

<p>By performing a grid search, the best parameters to pass into the Logistic Regression are identified for us. Below is the best estimator parameters for class weight and cost without scaling the data. The optimal values were estimated to be 0.01 for C (100 times smaller than the default 1.0 value) and 500 for max_iter (5 times greater than the default value of 100).

In [ ]:
#Diplay the top model parameters
RFestimator = regGridSearch.best_estimator_
RFestimator

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

regEstimator = XGBClassifier()


parameters = {
             }


#Create a grid search object using the  
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=regEstimator
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv_object # KFolds
                   , scoring='accuracy')

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X, y)

In [ ]:
#Diplay the top model parameters
XGBestimator = regGridSearch.best_estimator_
XGBestimator

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

# All

clf_array = [
    ('Logistic', LogisticRegression(C=0.01, class_weight='none', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=500,
          multi_class='ovr', n_jobs=1, penalty='l2',
          random_state=123456789, solver='lbfgs', tol=0.0001, verbose=0,
          warm_start=False)),
    ('Random Trees',       RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)),
    ('XGBoost',XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))
    ]

for clf in clf_array:
    acc = cross_val_score(clf[1],X,y)
    print (clf[0], acc.mean())

In [ ]:
# Train

for clf in clf_array:
    acc = cross_val_score(clf[1],X_train,y_train)
    print (clf[0], acc.mean())

In [ ]:
#Test

for clf in clf_array:
    acc = cross_val_score(clf[1],X_test,y_test)
    print (clf[0], acc.mean())

**Using SMOTE to perform over-sampling for balancing response**

In [ ]:
# Over-sampling using SMOTE to balance the NoShows
from imblearn.over_sampling import SMOTE
#Training set -- all records less than appointment date 5/1/2016
df_train = df.copy()
X_train = df_train

y_train = X_train['NoShow'].values # get the labels we want.
del X_train['NoShow'] # get rid of the class label

if 'NoShow' in X_train:
    del X_train['NoShow'] # get rid of the class label

sm = SMOTE()
X_train_ovr, y_train_ovr = sm.fit_sample(X_train, y_train.ravel())
print("Proportion of response in train set using SMOTE")
for i in np.unique(y_train) :
    print("The number of {} is {} accouting for {}%.".format(i, np.bincount(y_train)[i], np.round(np.bincount(y_train)[i]/len(y_train), 3)*100 ))


In [ ]:
from sklearn.linear_model import LogisticRegression
regEstimator = LogisticRegression()

parameters = { 'penalty':['l2']
              ,'C': [0.001, 0.01, 0.1, .5, 1, 10,100]
              ,'class_weight': [ 'none']
              ,'random_state': [123456789]
              ,'solver': ['lbfgs']
              ,'max_iter':[100,500,1000]
             }

#Create a grid search object using the  
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=regEstimator
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv_object # KFolds
                   , scoring='accuracy')

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X, y)

**Showing below that our time series is pretty well stationary over time except for Saturday which show consistantly lower no show appointments**

In [ ]:
df['NoShow'] = df['NoShow']
df['NoShow_num'] = np.where(df['NoShow']==True, 1, 0)

#Set dataframe with index as Appointment Day
ts = df[['AppointmentDayOrdinal', 'NoShow'] ]
ts = df.set_index('AppointmentDayOrdinal')
ts.groupby(['AppointmentDayOrdinal'])['NoShow'].sum()

**Plot below shows the # of NoShow appointmets over time.**

In [ ]:
plt.plot(ts.groupby(['AppointmentDayOrdinal'])['NoShow'].sum(), color="purple")

In [ ]:
regEstimator = LogisticRegression()
#Use the best parameters for our Linear Regression object
classifierEst = regGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics. 
EvaluateClassifierEstimator(classifierEst, X, y, cv_object)

#### Task 2 (Gender)

In [ ]:
#Testing set --- hold 1 month of data out as test set based on AppointmentDay
df_test = df.copy()

X_test = df_test[df['AppointmentDayOrdinal'] >=  dataSplit]
y_test = X_test['IsMale'].values # get the labels we want.

if 'IsMale' in X_test:
    del X_test['IsMale'] # get rid of the class label

#Training set -- all records less than appointment date 5/1/2016
df_train = df.copy() 
                 
X_train = df_train[df['AppointmentDayOrdinal'] < dataSplit]
y_train = X_train['IsMale'].values # get the labels we want.
del X_train['IsMale'] # get rid of the class label

if 'IsMale' in X_train:
    del X_train['IsMale'] # get rid of the class label

#All data
df_tmp = df.copy()
X = df_tmp
y = df_tmp['IsMale']
if 'IsMale' in X:
    del X['IsMale'] # get rid of the class label

In [ ]:
print ('Number of Prediciting Features: ', X.shape)
print ('Number of Response: ', y.shape)
print ('Appointment Date Range (Trainig Set): ', min(X_train['AppointmentDayOrdinal']) ," - ",  max(X_train['AppointmentDayOrdinal']))
print ('Number of Training Records: ', "Predictors: ", X_train.shape, "Response:" ,y_train.shape)
print ('Number of Test Records: ', "Predictors: ",X_test.shape, "Response:" ,y_test.shape)
print ('Appointment Date Range (Test Set): ', min(X_test['AppointmentDayOrdinal']) ," - ",  max(X_test['AppointmentDayOrdinal']))

In [ ]:
# Train


# lets train some trees
clf_array = [
    ('Logistic', LogisticRegression(C=0.01, class_weight='none', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=500,
          multi_class='ovr', n_jobs=1, penalty='l2',
          random_state=123456789, solver='lbfgs', tol=0.0001, verbose=0,
          warm_start=False)),
    ('Random Trees',RandomForestClassifier(max_depth=50, n_estimators=num_estimators)),
    ('XGBoost',XGBClassifier())
    ]

for clf in clf_array:
    acc = cross_val_score(clf[1],X_train,y_train)
    print (clf[0], acc.mean())

In [ ]:
#Test

num_estimators = 50
# lets train some trees
clf_array = [
    ('Logistic', LogisticRegression(C=0.01, class_weight='none', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=500,
          multi_class='ovr', n_jobs=1, penalty='l2',
          random_state=123456789, solver='lbfgs', tol=0.0001, verbose=0,
          warm_start=False)),
    ('Random Trees',RandomForestClassifier(max_depth=50, n_estimators=num_estimators)),
    ('XGBoost',XGBClassifier())
    ]

for clf in clf_array:
    acc = cross_val_score(clf[1],X_test,y_test)
    print (clf[0], acc.mean())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

# All Data
num_estimators = 50
#
clf_array = [
    ('Logistic', LogisticRegression(C=0.01, class_weight='none', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=500,
          multi_class='ovr', n_jobs=1, penalty='l2',
          random_state=123456789, solver='lbfgs', tol=0.0001, verbose=0,
          warm_start=False)),
    ('Random Trees',       RandomForestClassifier(max_depth=50, n_estimators=num_estimators)),
    ('XGBoost',XGBClassifier())
    ]

for clf in clf_array:
    acc = cross_val_score(clf[1],X,y)
    print (clf[0], acc.mean())

## Modeling and Evaluation 3

#### [20 points] Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!

**Meeting Notes**: 
**Luay (Recursive Feature Elimination for Logistic) here**
	1. Logistic Caroll to move from MiniLab 
	2. RandomForest - Chris
	3. XGBoost/Gradient - Caroll

Response Gender
**Caroll to split training/text with Gender variable**
**Luay (Recursive Feature Elimination for Logistic) here**
	1. Logistic Caroll to move from MiniLab
	2. Random -- Chris
	3. XGBoost/Gradient-- Caroll


#### Logistic Regression

In [ ]:

regEstimator = RandomForestClassifier()


parameters = { 'penalty':['l2']
              ,'C': [0.001, 0.01, 0.1, .5, 1, 10,100]
              ,'class_weight': [ 'none']
              ,'random_state': [123456789]
              ,'solver': ['lbfgs']
              ,'max_iter':[100,500,1000]
             }


#Create a grid search object using the  
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=regEstimator
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv_object # KFolds
                   , scoring='accuracy')

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X, y)
regGridSearch.best_estimator_

## Modeling and Evaluation 4

#### [10 points] Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.

**Meeting Notes**  Precision, recall, F-measure, ROC, AUC

## Modeling and Evaluation 5

#### [10 points] Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course

## Modeling and Evaluation 6

#### [10 points] Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.

In [ ]:
X_train.groupby(['AppointmentDay']).size()

In [ ]:
X_test.groupby(['AppointmentDay']).size()

## Deployment (5 points total) 

#### How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.? 

## Exceptional Work (10 points total) 

#### You have free reign to provide additional analyses. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?

**Meeting Notes** - We did this in MiniLab 1 with chart 
Possible to add Naives Bayes classification (Luay)